In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import pydub
from pydub.playback import play
import IPython.display as ipd
import os
output_notebook()

Loading BokehJS ...

In [58]:
# This class defines the core Guitar Effects object. 
# It contains functions to read and write audio files.
# It also contains all the different functions implementing various guitar effects

class GEcore():
    
    def __init__(self):
        self.effectname = ''
        self.audiofilename = ''
        self.framerate = []
        self.avgchannel = []
        self.read_audiofile()
        
    def read_audiofile(self):
        name = input('Enter the audio filename you want to read including the extension: ')
        filename, file_ext = os.path.splitext(name)
        filename = os.getcwd() + '/audiofiles/' + name
        self.audiofilename = filename
        audiofile = pydub.AudioSegment.from_file(filename, file_ext)
        audiofile = audiofile.fade_out(3000)
        songdata = []  # Empty list for holding audio data
        channels = []  # Empty list to hold data from separate channels
        songdata = np.frombuffer(audiofile._data, np.int16)
        for chn in range(audiofile.channels):
            channels.append(songdata[chn::audiofile.channels])  # separate signal from channels
        self.framerate = audiofile.frame_rate
        self.avgchannel = np.sum(channels, axis=0) / len(channels)  # Averaging signal over all channels
        p1 = self.plot_signal(self.avgchannel)
        show(p1)
        
    def plot_signal(self, asignal):
        p = figure(plot_width=900, plot_height=500, title='Audio Signal', 
            x_axis_label='Time (s)', y_axis_label='Amplitude (arb. units)')
        time = np.linspace(0, len(asignal)/self.framerate, len(asignal))
        p.line(time[0::500], asignal[0::500])
        return(p)
    
    def delay(self, input_signal):
        delaytime = int(input('Enter the delay you want to add in miliseconds: '))
        gain = float(input('Enter the gain (a number betweeen 0 and 1): '))
        num = int(delaytime * 1e-3 * self.framerate)
        shifted_chan = np.copy(input_signal)
        shifted_chan[:num] = 0
        channel_delay = input_signal + gain * shifted_chan
        p2 = self.plot_signal(channel_delay)
        show(p2)
        return channel_delay

In [59]:
ge = GEcore()

Enter the audio filename you want to read including the extension: cello-phrase.wav


In [60]:
ipd.Audio(ge.audiofilename, rate=ge.framerate)

In [61]:
sound_delay = ge.delay(ge.avgchannel)

Enter the delay you want to add in miliseconds: 3000
Enter the gain (a number betweeen 0 and 1): 1


In [62]:
ipd.Audio(sound_delay, rate=ge.framerate)

In [63]:
p = figure(plot_width=900, plot_height=500, title='Audio Signal', 
            x_axis_label='Time (s)', y_axis_label='Amplitude (arb. units)')
time = np.linspace(0, len(ge.avgchannel)/ge.framerate, len(ge.avgchannel))
p.line(time[0::500], ge.avgchannel[0::500], color='red')
p.line(time[0::500], sound_delay[0::500])
show(p)